In [1]:
from bs4 import BeautifulSoup
import urllib3
from urllib.parse import urljoin
import re
import nltk

# Crawler das urls

In [2]:
def crawl(paginas, profundidade):
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    for i in range(profundidade):
        novas_paginas = set()
        for pagina in paginas:
            http = urllib3.PoolManager()
            try:
                dados_pagina = http.request('GET', pagina)
            except:
                print('Erro na página ' + pagina)
                continue
            
            sopa = BeautifulSoup(dados_pagina.data, 'html')
            indexador(pagina, sopa)
            links = sopa.find_all('a')
            for link in links:
                if 'href' in link.attrs:
                    url = urljoin(pagina, str(link.get('href')))
                    if url.find("'") != -1:
                        continue
                    url = url.split('#')[0]
                    if url[0:4] == 'http':
                        novas_paginas.add(url)
                    
            paginas = novas_paginas
    return paginas

In [32]:
teste = crawl(['https://pt.wikipedia.org/wiki/Linguagem_de_programa%C3%A7%C3%A3o'], 2)

/usr/local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Erro na página https://wikimediafoundation.org/wiki/Condições_de_Uso
Erro na página https://be-x-old.wikipedia.org/wiki/%D0%9C%D0%BE%D0%B2%D0%B0_%D0%BF%D1%80%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%B0%D0%B2%D0%B0%D0%BD%D1%8C%D0%BD%D1%8F
Erro na página https://pt.wikipedia.org/w/index.php?title=Linguagem_de_programação&oldid=50743371


# Pré-processamento dos textos: remoção das tags html

In [3]:
# remover algumas tags
def getTexto(sopa):
    for tag in sopa(['script', 'style']):
        tag.decompose()
    
    return ' '.join(sopa.stripped_strings)

# Pré-processamento dos textos: separacao das palavras

In [4]:
def separaPalavras(texto):
    splitter = re.compile('\\W*')
    stop = nltk.corpus.stopwords.words('portuguese')
    stemmer = nltk.stem.RSLPStemmer()
    lista_palavras = []
    lista = [p for p in splitter.split(texto) if p != '' ] 
    for p in lista:
        if p.lower() not in stop and len(p) > 1:
            lista_palavras.append(stemmer.stem(p).lower())
            
    return lista_palavras

In [19]:
separaPalavras('Este lugar é apavorante')

/home/cloves/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: FutureWarning: split() requires a non-empty pattern match.
  


['lug', 'apavor']

# Conexão com BD

In [6]:
import os
import psycopg2

In [7]:
DB = os.environ["NOME_DO_MEU_DB"] 
HOST = os.environ["HOST_DO_MEU_DB"] 
USER = os.environ["USUARIO_DO_MEU_DB"]
PASS = os.environ["SENHA_DO_MEU_DB"]

In [15]:
def conectar():
    conn_string = "host='%s' dbname='%s' user='%s' password='%s'" %(HOST, DB, USER, PASS)
    try:
        # get a connection, if a connect cannot be made an exception will be raised here
        conn = psycopg2.connect(conn_string)
        conn.set_client_encoding('UNICODE')
        return conn
    except:
        print ("Can't connected to database!")

In [16]:
con = conectar()

In [5]:
con.close()

# Verificação de páginas já indexadas

In [9]:
def paginaIndexada(url):
    retorno = -1 # não exite página
    con  = conectar()
    cursorURL = con.cursor()
    cursorURL.execute("select idurl from buscas_em_textos.urls where url = '%s'" %url)
    if cursorURL.rowcount > 0:
        #print('URL já cadastrada!')
        idurl = cursorURL.fetchone()[0]
        cursorPalava = con.cursor()
        cursorPalava.execute("select idurl from buscas_em_textos.palavra_localizacao \
                             where idurl = '%s'" %idurl)
        if cursorPalava.rowcount > 0:
            #print('Url com palavras')
            retorno = -2 # existe a página com palavras cadastradas
        else:
            #print('URL sem palavras')
            retorno = idurl # existe a página sem palavras
        cursorPalava.close()
    #else:
       # print ('URL não cadastrada!')
        
    cursorURL.close()
    con.close()
    
    return retorno

In [7]:
paginaIndexada('www.cloves.com')

Connected to database!


-1

# Inclusão de URLs no índice

In [19]:
def inserPagina(url):
    con  = conectar()
    cursor = con.cursor()
    cursor.execute("select idurl from buscas_em_textos.urls\
                    order by idurl desc limit 1")
    try:
        idpagina = cursor.fetchall()[0][0] + 1
    except:
        idpagina = 0
    cursor.execute("insert into buscas_em_textos.urls (idurl, url) values(%d,'%s')" %(idpagina,url))
    con.commit()
    cursor.close()
    con.close()
    return idpagina

In [68]:
inserPagina('www.teste4.com')

Connected to database!


3

# Verificação de palavras já indexadas 

In [11]:
def palavraIndexada(palavra):
    retorno = -1 # não existe a palavra no índice
    con  = conectar()
    cursor = con.cursor()
    cursor.execute("select idpalavra from buscas_em_textos.palavras where palavra = '%s'" %palavra)
    if cursor.rowcount > 0:
        #print('palavra já cadastrada')
        retorno = cursor.fetchone()[0]
    #else:
        #print('palavra não cadastrada')
    cursor.close()
    con.close()
    return retorno

In [74]:
palavraIndexada('cloves')

-1

# Inclusão de Palavras no Índice

In [20]:
def inserePalavra(palavra):
    con  = conectar()
    cursor = con.cursor()
    cursor.execute("select idpalavra from buscas_em_textos.palavras \
                    order by idpalavra desc limit 1")
    
    try:
        idpalavra = cursor.fetchall()[0][0] + 1
    except:
        idpalavra = 0
        
    cursor.execute("insert into buscas_em_textos.palavras (idpalavra, palavra) \
                    values(%d, '%s')" %(idpalavra, palavra))
    
    con.commit()
    cursor.close()
    con.close()
    return idpalavra

In [85]:
inserePalavra('teste2')

1

# Inclusão das palavras do documento

In [21]:
def inserPalavraLocalizacao(idurl, idpalavra, localizacao):
    con  = conectar()
    cursor = con.cursor()
    cursor.execute("select idpalavra_localizacao from buscas_em_textos.palavra_localizacao \
                    order by idpalavra_localizacao desc limit 1")
    try:
        idpalavra_localizacao = cursor.fetchall()[0][0] + 1
    except:
        idpalavra_localizacao = 0
        
    query = "insert into buscas_em_textos.palavra_localizacao \
            (idpalavra_localizacao, idurl, idpalavra, localizacao) \
            values(%d, %d, %d, %d)" %(idpalavra_localizacao,idurl,idpalavra,localizacao)
    
    cursor.execute(query)
    con.commit()
    cursor.close()
    con.close()
    return idpalavra_localizacao

In [91]:
inserPalavraLocalizacao(0, 1, 2)

1

# Indexador Completo

In [14]:
def indexador(url, sopa):
    indexada = paginaIndexada(url)
    if indexada == -2:
        print("URL já indexada!")
        return
    elif indexada == -1:
        idnova_pagina = inserPagina(url)
    elif indexada > 0:
        idnova_pagina = indexada
    
    print("Indexando " + url)
    
    texto = getTexto(sopa)
    palavras = separaPalavras(texto)
    for i in range(len(palavras)):
        palavra = palavras[i]
        idpalavra = palavraIndexada(palavra)
        if idpalavra == -1:
            idpalavra = inserePalavra(palavra)
        inserPalavraLocalizacao(idnova_pagina, idpalavra, i)